In [1]:
import os

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim

In [2]:
os.chdir("../")

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2,   
                           saturation=0.2, hue=0.1),

     transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225]),

])                     

In [4]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225]),
])

### Dataset Load

In [5]:
TRAINING_DIR = "data/processed/train"
VALIDATION_DIR = "data/processed/test"

In [6]:
train_data = ImageFolder(root=TRAINING_DIR, transform= train_transform)
test_data = ImageFolder(root=VALIDATION_DIR, transform=test_transform)

In [7]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=1)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=1)

In [8]:
batches = len(train_loader)

In [9]:
for i, (images, labels) in enumerate(train_loader):
    print(f"Batch {i+1}:")
    print(f"Images shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    if i == 2:  # Display only the first 3 batches
        break

Batch 1:
Images shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])
Batch 2:
Images shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])
Batch 3:
Images shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])


In [10]:
# Get class names and number of classes
class_names = train_data.classes
num_classes = len(class_names)
print(f'Number of classes: {num_classes}')
print(f'Classes: {class_names}')


Number of classes: 14
Classes: ['bbq', 'biryani', 'brownie', 'butter_chicken', 'chai', 'chapati', 'chicken_tikka', 'french_fries', 'fried_rice', 'haleem', 'omelette', 'paratha', 'paratha_roll', 'samosa']


### Model Architecture

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(512 * 14 * 14, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.relu(self.pool(self.conv1(x)))
        
        x = self.relu(self.pool(self.conv2(x)))
        x = self.relu(self.pool(self.conv3(x)))
        x = self.relu(self.pool(self.conv4(x)))
        x = x.view(-1, 512 * 14 * 14)  #
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [12]:
model = CNN()

In [13]:
model.parameters

<bound method Module.parameters of CNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (fc1): Linear(in_features=100352, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=14, bias=True)
)>

In [14]:
LEARNING_RATE = 0.001

In [15]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [16]:
criterion = nn.CrossEntropyLoss()

In [17]:
EPOCHS = 10

### Training Loop

In [18]:
import torch

In [19]:
torch.cuda.is_available()

False

In [20]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(DEVICE)

In [1]:
def training_loop():
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch,(image, label) in enumerate(train_loader):
        image = image.to(torch.device(DEVICE))
        label = label.to(torch.device(DEVICE))
        
        optimizer.zero_grad()
        
        # Forward Pass
        output = model(image)
        
        loss = criterion(output, label)
        
        # Backward Pass
        loss.backward()
        
        optimizer.step()


        running_loss += loss.item()

        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        
        if batch % 5 == 0:
            print(f'Batch {batch}/{len(train_loader)}, Loss: {loss.item():.4f}')
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

In [ ]:
def validation_loop():
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch, (image, label) in enumerate(test_loader):
            image = image.to(torch.device(DEVICE))
            label = label.to(torch.device(DEVICE))
            
            output = model(image)
            loss = criterion(output, label)
            
            running_loss += loss.item()
            
            _, predicted = torch.max(output.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()

            if batch % 5 == 0:
                print(f'Batch {batch}/{len(train_loader)}, Loss: {loss.item():.4f}')
    
    
    epoch_loss = running_loss / len(test_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

In [ ]:
def epoch_loop(EPOCHS):
    for epoch in range (EPOCHS):
        running_loss = 0.0
        correct = 0
        total = 0
        print(f"Epoch {epoch+1}/{EPOCHS}")
        for batch,(image, label) in enumerate(train_loader):
            image = image.to(torch.device(DEVICE))
            label = label.to(torch.device(DEVICE))
            
            optimizer.zero_grad()
            
            # Forward Pass
            output = model(image)
            
            loss = criterion(output, label)
            
            # Backward Pass
            loss.backward()
            
            optimizer.step()


            running_loss += loss.item()

            _, predicted = torch.max(output.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
            
            if batch % 5 == 0:
                print(f'Batch {batch}/{len(train_loader)}, Loss: {loss.item():.4f}')
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100. * correct / total
        
        print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

Epoch 1/10
Batch 0/33, Loss: 2.6367
Batch 5/33, Loss: 2.6387
Batch 10/33, Loss: 2.6354
Batch 15/33, Loss: 2.6366
Batch 20/33, Loss: 2.6412
Batch 25/33, Loss: 2.6426
Batch 30/33, Loss: 2.6390
Epoch 1/10, Loss: 2.6388, Accuracy: 8.38%
Epoch 2/10
Batch 0/33, Loss: 2.6340
Batch 5/33, Loss: 2.6365
Batch 10/33, Loss: 2.6375
Batch 15/33, Loss: 2.6385
Batch 20/33, Loss: 2.6345
Batch 25/33, Loss: 2.6341
Batch 30/33, Loss: 2.6346
Epoch 2/10, Loss: 2.6365, Accuracy: 9.62%
Epoch 3/10
Batch 0/33, Loss: 2.6341
Batch 5/33, Loss: 2.6343
Batch 10/33, Loss: 2.6364
Batch 15/33, Loss: 2.6369
Batch 20/33, Loss: 2.6363
Batch 25/33, Loss: 2.6330
Batch 30/33, Loss: 2.6364
Epoch 3/10, Loss: 2.6347, Accuracy: 11.43%
Epoch 4/10
Batch 0/33, Loss: 2.6394
Batch 5/33, Loss: 2.6355
Batch 10/33, Loss: 2.6363
Batch 15/33, Loss: 2.6285
Batch 20/33, Loss: 2.6361
Batch 25/33, Loss: 2.6285
Batch 30/33, Loss: 2.6319
Epoch 4/10, Loss: 2.6328, Accuracy: 12.48%
Epoch 5/10
Batch 0/33, Loss: 2.6256
Batch 5/33, Loss: 2.6273
Batch